In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete

Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  
0                 66          79          0  District  2917  1910635  
1                 94          61          0  District  2917  1910635  
2                 90          60          0  District  2917  1910635  
3                 67          58          0  District  2917  1910635  
4                 97          84          0  District  2917  1910635  
...              ...         ...        ...       ...   ...      ...  
39165             99          90         14   Charter  1635  1043130  
39166             95          70         14   Charter  1635  1043130  
39167             73          84         14   Charter  1635  1043130  
39168             99          90         14   Charter  1635  1043130  
39169             95          75         14   Charter  1635  1043130  

[39170 rows x 11 columns]

In [2]:
# Count the number of unique school names
school_count = len(school_data_complete["school_name"].unique())

# Count the total number of students
student_count = school_data_complete["student_name"].count()

#Grab one budget value for each unique school and sum those 
school_list = school_data_complete["school_name"].unique()
schools_sample = pd.DataFrame()
for school in school_list:
    schools_sample=schools_sample.append(school_data_complete[school_data_complete["school_name"] == school].
                                         sample(1)) 
total_budget = schools_sample["budget"].sum()

#get mean math and reading score and calculate math, reading, and overall passing percentages
mean_math = school_data_complete["math_score"].mean()
mean_reading = school_data_complete["reading_score"].mean()
pass_math_pct = len(school_data_complete[school_data_complete["math_score"] >= 70])/student_count

pass_reading_pct = len(school_data_complete[school_data_complete["reading_score"] >= 70])/student_count

pass_OA_pct = len(school_data_complete[(school_data_complete["math_score"] >= 70) & 
                                       (school_data_complete["reading_score"] >= 70)])/student_count

#create a new dataframe with the summarized data
summary_df = pd.DataFrame(
    {"Total Schools": [school_count],
     "Total Students": [student_count],
     "Total Budget": [total_budget], 
     "Average Math Score": [mean_math], 
     "Average Reading Score": [mean_reading],
     "% Passing Math": [pass_math_pct], 
     "% Passing Reading": [pass_reading_pct], 
     "% Overall Passing": [pass_OA_pct]})

#Format the data for a clean appearance 
summary_df_formatted = summary_df
summary_df_formatted["Total Students"] = summary_df_formatted["Total Students"].map("{:,}".format)
summary_df_formatted["Total Budget"] = summary_df_formatted["Total Budget"].map("${:,.2f}".format)
summary_df_formatted["% Passing Math"] = summary_df_formatted["% Passing Math"].map("{:.2%}".format)
summary_df_formatted["% Passing Reading"] = summary_df_formatted["% Passing Reading"].map("{:.2%}".format)
summary_df_formatted["% Overall Passing"] = summary_df_formatted["% Overall Passing"].map("{:.2%}".format)

summary_df_formatted

Total Schools Total Students    Total Budget  Average Math Score  \
0             15         39,170  $24,649,428.00           78.985371   

   Average Reading Score % Passing Math % Passing Reading % Overall Passing  
0               81.87784         74.98%            85.81%            65.17%

In [3]:

# group the complete school data by school name and aggregate all columns where a simple aggregation will suffice
# Note: I used max to pick just one of the values that were repeated across all rows for a school
# (budget, type)
grouped_schools = school_data_complete.groupby(['school_name']).\
                agg({"type": ["max"], 
                     "Student ID": ["count"], 
                     "budget": ["max"],
                     "math_score": ["mean"],
                     "reading_score": ["mean"]})
                    
# Drop the aggregate functions from the column names (to me, this makes the column names easier to deal with)
grouped_schools.columns = grouped_schools.columns.droplevel(1)

# Create a subset of the original data with only students that passed math and group by school
subset_df_math = school_data_complete[(school_data_complete["math_score"] >= 70)] 
passed_math_df = subset_df_math.groupby(['school_name'])

# Create a subset of the original data with only students that passed reading and group by school
subset_df_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]  
passed_reading_df = subset_df_reading.groupby(['school_name'])

# Create a subset of the original data with only students that passed both and group by school
subset_df_both = school_data_complete[(school_data_complete["math_score"] >= 70) &
                                      (school_data_complete["reading_score"] >= 70)]
passed_both_df = subset_df_both.groupby(['school_name'])

# Strip unneeded columns from "both" subset and rename the remaining column
passed_both_df = passed_both_df.count()
passed_both_df = passed_both_df.drop(passed_both_df.columns[[ 1, 2, 3, 4, 5, 6, 7, 8, 9]], axis=1)
passed_both_df = passed_both_df.rename(columns = {"Student ID": "% Overall Passing"})

# Strip unneeded columns from "math" subset and rename the remaining column
passed_math_df = passed_math_df.count()
passed_math_df = passed_math_df.drop(passed_math_df.columns[[ 1, 2, 3, 4, 5, 6, 7, 8, 9]], axis=1)
passed_math_df = passed_math_df.rename(columns = {"Student ID": "% Passing Math"})

# Strip unneeded columns from "reading" subset and rename the remaining column
passed_reading_df = passed_reading_df.count()
passed_reading_df = passed_reading_df.drop(passed_reading_df.columns[[ 1, 2, 3, 4, 5, 6, 7, 8, 9]], axis=1)
passed_reading_df = passed_reading_df.rename(columns = {"Student ID": "% Passing Reading"})

# Merge all 4 data frames on "school name"
new_df = pd.merge(grouped_schools, passed_math_df, how="left", on="school_name")
new_df = pd.merge(new_df, passed_reading_df, how="left", on="school_name") 
new_df = pd.merge(new_df, passed_both_df, how="left", on="school_name") 

# Divide passing counts by student total to get percentages
new_df["% Passing Math"] = (new_df["% Passing Math"]/new_df["Student ID"])
new_df["% Passing Reading"] = (new_df["% Passing Reading"]/new_df["Student ID"])
new_df["% Overall Passing"] = (new_df["% Overall Passing"]/new_df["Student ID"])

# Create a new column for per student budget
new_df["Per Student Budget"] = (new_df["budget"]/new_df["Student ID"])

# Rename the columns
new_df = new_df.rename(columns = {"type": "School Type",
                                                 "Student ID": "Total Students",
                                                 "budget": "Total School Budget",
                                                 "math_score": "Average Math Score",
                                                 "reading_score": "Average Reading Score",})
                                                
# Re-order the columns
new_df = new_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                 "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                 "% Overall Passing"]]


# Make a copy of the dataframe for formatting so that the original can still be used for later calculations
data_by_school = new_df.copy()

# Format the copied dataframe
data_by_school["Total Students"] = data_by_school["Total Students"].map("{:,}".format)
data_by_school["Total School Budget"] = data_by_school["Total School Budget"].map("${:,.2f}".format)
data_by_school["Per Student Budget"] = data_by_school["Per Student Budget"].map("${:,.2f}".format)
data_by_school["Average Reading Score"] = data_by_school["Average Reading Score"].map("{:,.2f}".format)
data_by_school["Average Math Score"] = data_by_school["Average Math Score"].map("{:,.2f}".format)
data_by_school["% Passing Math"] = data_by_school["% Passing Math"].map("{:.2%}".format)
data_by_school["% Passing Reading"] = data_by_school["% Passing Reading"].map("{:.2%}".format)
data_by_school["% Overall Passing"] = data_by_school["% Overall Passing"].map("{:.2%}".format)

data_by_school

School Type Total Students Total School Budget  \
school_name                                                            
Bailey High School       District          4,976       $3,124,928.00   
Cabrera High School       Charter          1,858       $1,081,356.00   
Figueroa High School     District          2,949       $1,884,411.00   
Ford High School         District          2,739       $1,763,916.00   
Griffin High School       Charter          1,468         $917,500.00   
Hernandez High School    District          4,635       $3,022,020.00   
Holden High School        Charter            427         $248,087.00   
Huang High School        District          2,917       $1,910,635.00   
Johnson High School      District          4,761       $3,094,650.00   
Pena High School          Charter            962         $585,858.00   
Rodriguez High School    District          3,999       $2,547,363.00   
Shelton High School       Charter          1,761       $1,056,600.00   
Thomas High School        Charter          1,635       $1,043,130.00   
Wilson High School        Charter          2,283       $1,319,574.00   
Wright High School        Charter          1,800       $1,049,400.00   

                      Per Student Budget Average Math Score  \
school_name                                                   
Bailey High School               $628.00              77.05   
Cabrera High School              $582.00              83.06   
Figueroa High School             $639.00              76.71   
Ford High School                 $644.00              77.10   
Griffin High School              $625.00              83.35   
Hernandez High School            $652.00              77.29   
Holden High School               $581.00              83.80   
Huang High School                $655.00              76.63   
Johnson High School              $650.00              77.07   
Pena High School                 $609.00              83.84   
Rodriguez High School            $637.00              76.84   
Shelton High School              $600.00              83.36   
Thomas High School               $638.00              83.42   
Wilson High School               $578.00              83.27   
Wright High School               $583.00              83.68   

                      Average Reading Score % Passing Math % Passing Reading  \
school_name                                                                    
Bailey High School                    81.03         66.68%            81.93%   
Cabrera High School                   83.98         94.13%            97.04%   
Figueroa High School                  81.16         65.99%            80.74%   
Ford High School                      80.75         68.31%            79.30%   
Griffin High School                   83.82         93.39%            97.14%   
Hernandez High School                 80.93         66.75%            80.86%   
Holden High School                    83.81         92.51%            96.25%   
Huang High School                     81.18         65.68%            81.32%   
Johnson High School                   80.97         66.06%            81.22%   
Pena High School                      84.04         94.59%            95.95%   
Rodriguez High School                 80.74         66.37%            80.22%   
Shelton High School                   83.73         93.87%            95.85%   
Thomas High School                    83.85         93.27%            97.31%   
Wilson High School                    83.99         93.87%            96.54%   
Wright High School                    83.95         93.33%            96.61%   

                      % Overall Passing  
school_name                              
Bailey High School               54.64%  
Cabrera High School              91.33%  
Figueroa High School             53.20%  
Ford High School                 54.29%  
Griffin High School              90.60%  
Hernandez High School            53.53%  
Holden High School          

In [4]:
#Sort descending by overall passing

top_data_by_school = data_by_school.sort_values(by=['% Overall Passing'],ascending=False)
top_data_by_school.head(5)

School Type Total Students Total School Budget  \
school_name                                                          
Cabrera High School     Charter          1,858       $1,081,356.00   
Thomas High School      Charter          1,635       $1,043,130.00   
Griffin High School     Charter          1,468         $917,500.00   
Wilson High School      Charter          2,283       $1,319,574.00   
Pena High School        Charter            962         $585,858.00   

                    Per Student Budget Average Math Score  \
school_name                                                 
Cabrera High School            $582.00              83.06   
Thomas High School             $638.00              83.42   
Griffin High School            $625.00              83.35   
Wilson High School             $578.00              83.27   
Pena High School               $609.00              83.84   

                    Average Reading Score % Passing Math % Passing Reading  \
school_name                                                                  
Cabrera High School                 83.98         94.13%            97.04%   
Thomas High School                  83.85         93.27%            97.31%   
Griffin High School                 83.82         93.39%            97.14%   
Wilson High School                  83.99         93.87%            96.54%   
Pena High School                    84.04         94.59%            95.95%   

                    % Overall Passing  
school_name                            
Cabrera High School            91.33%  
Thomas High School             90.95%  
Griffin High School            90.60%  
Wilson High School             90.58%  
Pena High School               90.54%

In [5]:
#Sort ascending by overall passing

bottom_data_by_school = data_by_school.sort_values(by=['% Overall Passing'],ascending=True)
bottom_data_by_school.head(5)

School Type Total Students Total School Budget  \
school_name                                                            
Rodriguez High School    District          3,999       $2,547,363.00   
Figueroa High School     District          2,949       $1,884,411.00   
Huang High School        District          2,917       $1,910,635.00   
Hernandez High School    District          4,635       $3,022,020.00   
Johnson High School      District          4,761       $3,094,650.00   

                      Per Student Budget Average Math Score  \
school_name                                                   
Rodriguez High School            $637.00              76.84   
Figueroa High School             $639.00              76.71   
Huang High School                $655.00              76.63   
Hernandez High School            $652.00              77.29   
Johnson High School              $650.00              77.07   

                      Average Reading Score % Passing Math % Passing Reading  \
school_name                                                                    
Rodriguez High School                 80.74         66.37%            80.22%   
Figueroa High School                  81.16         65.99%            80.74%   
Huang High School                     81.18         65.68%            81.32%   
Hernandez High School                 80.93         66.75%            80.86%   
Johnson High School                   80.97         66.06%            81.22%   

                      % Overall Passing  
school_name                              
Rodriguez High School            52.99%  
Figueroa High School             53.20%  
Huang High School                53.51%  
Hernandez High School            53.53%  
Johnson High School              53.54%

In [6]:
#create the 9th grade average math scores by school
subset_9th_math = school_data_complete[(school_data_complete["grade"] == "9th")] 
mean_9th_math = subset_9th_math.groupby(['school_name'])
mean_9th_math = mean_9th_math["math_score"].mean()

#create the 10th grade average math scores by school
subset_10th_math = school_data_complete[(school_data_complete["grade"] == "10th")] 
mean_10th_math = subset_10th_math.groupby(['school_name'])
mean_10th_math = mean_10th_math["math_score"].mean()

#create the 11th grade average math scores by school
subset_11th_math = school_data_complete[(school_data_complete["grade"] == "11th")] 
mean_11th_math = subset_11th_math.groupby(['school_name'])
mean_11th_math = mean_11th_math["math_score"].mean()

#create the 12th grade average math scores by school
subset_12th_math = school_data_complete[(school_data_complete["grade"] == "12th")] 
mean_12th_math = subset_12th_math.groupby(['school_name'])
mean_12th_math = mean_12th_math["math_score"].mean()

#Created the data frame containing all four grades
math_by_grade_df = pd.DataFrame(
        {"9th":mean_9th_math,
         "10th":mean_10th_math,
         "11th":mean_11th_math, 
         "12th":mean_12th_math})
         
math_by_grade_df         

9th       10th       11th       12th
school_name                                                      
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

In [7]:
#create the 9th grade average reading scores by school
subset_9th_reading = school_data_complete[(school_data_complete["grade"] == "9th")] 
mean_9th_reading = subset_9th_reading.groupby(['school_name'])
mean_9th_reading = mean_9th_reading["reading_score"].mean()

#create the 10th grade average reading scores by school
subset_10th_reading = school_data_complete[(school_data_complete["grade"] == "10th")] 
mean_10th_reading = subset_10th_reading.groupby(['school_name'])
mean_10th_reading = mean_10th_reading["reading_score"].mean()

#create the 11th grade average reading scores by school
subset_11th_reading = school_data_complete[(school_data_complete["grade"] == "11th")] 
mean_11th_reading = subset_11th_reading.groupby(['school_name'])
mean_11th_reading = mean_11th_reading["reading_score"].mean()

#create the 12th grade average reading scores by school
subset_12th_reading = school_data_complete[(school_data_complete["grade"] == "12th")] 
mean_12th_reading = subset_12th_reading.groupby(['school_name'])
mean_12th_reading = mean_12th_reading["reading_score"].mean()

#Created the data frame containing all four grades
reading_by_grade_df = pd.DataFrame(
        {"9th":mean_9th_reading,
         "10th":mean_10th_reading,
         "11th":mean_11th_reading, 
         "12th":mean_12th_reading})
         
reading_by_grade_df  

9th       10th       11th       12th
school_name                                                      
Bailey High School     81.303155  80.907183  80.945643  80.912451
Cabrera High School    83.676136  84.253219  83.788382  84.287958
Figueroa High School   81.198598  81.408912  80.640339  81.384863
Ford High School       80.632653  81.262712  80.403642  80.662338
Griffin High School    83.369193  83.706897  84.288089  84.013699
Hernandez High School  80.866860  80.660147  81.396140  80.857143
Holden High School     83.677165  83.324561  83.815534  84.698795
Huang High School      81.290284  81.512386  81.417476  80.305983
Johnson High School    81.260714  80.773431  80.616027  81.227564
Pena High School       83.807273  83.612000  84.335938  84.591160
Rodriguez High School  80.993127  80.629808  80.864811  80.376426
Shelton High School    84.122642  83.441964  84.373786  82.781671
Thomas High School     83.728850  84.254157  83.585542  83.831361
Wilson High School     83.939778  84.021452  83.764608  84.317673
Wright High School     83.833333  83.812757  84.156322  84.073171

In [8]:
# copy the dataframe that was aggregated by school from above
temp_df = new_df.copy()

# Weight the scores of interest by school size. This will be needed to calculate weighted averages later.
temp_df["weighted_math"] = (temp_df["Average Math Score"]*temp_df["Total Students"])
temp_df["weighted_reading"] = (temp_df["Average Reading Score"]*temp_df["Total Students"])
temp_df["weighted_math_pct"] = (temp_df["% Passing Math"]*temp_df["Total Students"])
temp_df["weighted_reading_pct"] = (temp_df["% Passing Reading"]*temp_df["Total Students"])
temp_df["weighted_overall_pct"] = (temp_df["% Overall Passing"]*temp_df["Total Students"])

#Create appropriate bins and cut the data by Per Student Budget
bins = [0, 595, 616, 637, 658]
group_names = ["<$595", "$595-616", "$616-637", "$637-658"]

new_col = pd.cut(temp_df["Per Student Budget"], bins, labels=group_names, include_lowest=True)
temp_df["Spending Ranges (Per Student)"] = new_col

#Create appropriate bins and cut the data by Total Students
bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

new_col = pd.cut(temp_df["Total Students"], bins, labels=group_names, include_lowest=True)
temp_df["School Size"] = new_col


#This Data will be used in the next three cells to 


In [9]:
# Group By Spending Per Student Using the Weighted values (temp_df) from a previous cell

temp_df_spending = temp_df.groupby(['Spending Ranges (Per Student)']).\
                        agg({"Total Students": ["sum"], 
                             "weighted_math": ["sum"], 
                             "weighted_reading": ["sum"], 
                             "weighted_math_pct": ["sum"],
                             "weighted_reading_pct": ["sum"], 
                             "weighted_overall_pct": ["sum"]}) 

#drop the aggregating function from the column name
temp_df_spending.columns = temp_df_spending.columns.droplevel(1)

#divide the weighted sums by the total students in each group to get the weighted averages
temp_df_spending["weighted_math"] = (temp_df_spending["weighted_math"]/temp_df_spending["Total Students"])
temp_df_spending["weighted_reading"] = (temp_df_spending["weighted_reading"]/temp_df_spending["Total Students"])
temp_df_spending["weighted_math_pct"] = (temp_df_spending["weighted_math_pct"]/temp_df_spending["Total Students"])
temp_df_spending["weighted_reading_pct"] = (temp_df_spending["weighted_reading_pct"]/temp_df_spending["Total Students"])
temp_df_spending["weighted_overall_pct"] = (temp_df_spending["weighted_overall_pct"]/temp_df_spending["Total Students"])
    
#drop the total students column as it is no longer needed
temp_df_spending = temp_df_spending.drop(temp_df_spending.columns[[ 0 ]], axis=1)

#rename the columns
temp_df_spending = temp_df_spending.rename(columns = {"weighted_math": "Average Math Score",
                                                 "weighted_reading": "Average Reading Score",
                                                 "weighted_math_pct": "% Passing Math",
                                                 "weighted_reading_pct": "% Passing Reading",
                                                 "weighted_overall_pct": "% Overall Passing",})
    
#copy the data frame to a new one for formatting
spending_group_formatted = temp_df_spending.copy()

#format the copied data frame
spending_group_formatted["Average Reading Score"] = \
                                        spending_group_formatted["Average Reading Score"].map("{:,.2f}".format)
spending_group_formatted["Average Math Score"] = spending_group_formatted["Average Math Score"].map("{:,.2f}".format)
spending_group_formatted["% Passing Math"] = spending_group_formatted["% Passing Math"].map("{:.2%}".format)
spending_group_formatted["% Passing Reading"] = spending_group_formatted["% Passing Reading"].map("{:.2%}".format)
spending_group_formatted["% Overall Passing"] = spending_group_formatted["% Overall Passing"].map("{:.2%}".format)

spending_group_formatted

Average Math Score Average Reading Score  \
Spending Ranges (Per Student)                                            
<$595                                      83.36                 83.96   
$595-616                                   83.53                 83.84   
$616-637                                   77.86                 81.31   
$637-658                                   77.54                 81.23   

                              % Passing Math % Passing Reading  \
Spending Ranges (Per Student)                                    
<$595                                 93.70%            96.69%   
$595-616                              94.12%            95.89%   
$616-637                              70.32%            83.41%   
$637-658                              68.74%            82.15%   

                              % Overall Passing  
Spending Ranges (Per Student)                    
<$595                                    90.64%  
$595-616                                 90.12%  
$616-637                                 59.06%  
$637-658                                 56.70%

In [10]:
# Group By School Size Using the Weighted values (temp_df) from a previous cell

temp_df_size = temp_df.groupby(['School Size']).\
                        agg({"Total Students": ["sum"], 
                             "weighted_math": ["sum"], 
                             "weighted_reading": ["sum"], 
                             "weighted_math_pct": ["sum"],
                             "weighted_reading_pct": ["sum"], 
                             "weighted_overall_pct": ["sum"]}) 

#drop the aggregating function from the column name
temp_df_size.columns = temp_df_size.columns.droplevel(1)

#divide the weighted sums by the total students in each group to get the weighted averages
temp_df_size["weighted_math"] = (temp_df_size["weighted_math"]/temp_df_size["Total Students"])
temp_df_size["weighted_reading"] = (temp_df_size["weighted_reading"]/temp_df_size["Total Students"])
temp_df_size["weighted_math_pct"] = (temp_df_size["weighted_math_pct"]/temp_df_size["Total Students"])
temp_df_size["weighted_reading_pct"] = (temp_df_size["weighted_reading_pct"]/temp_df_size["Total Students"])
temp_df_size["weighted_overall_pct"] = (temp_df_size["weighted_overall_pct"]/temp_df_size["Total Students"])
    
#drop the total students column as it is no longer needed
temp_df_size = temp_df_size.drop(temp_df_size.columns[[ 0 ]], axis=1)

#rename the columns
temp_df_size = temp_df_size.rename(columns = {"weighted_math": "Average Math Score",
                                                 "weighted_reading": "Average Reading Score",
                                                 "weighted_math_pct": "% Passing Math",
                                                 "weighted_reading_pct": "% Passing Reading",
                                                 "weighted_overall_pct": "% Overall Passing",})
    
#copy the data frame to a new one for formatting
size_group_formatted = temp_df_size.copy()

#format the copied data frame
size_group_formatted["Average Reading Score"] = \
                                        size_group_formatted["Average Reading Score"].map("{:,.2f}".format)
size_group_formatted["Average Math Score"] = size_group_formatted["Average Math Score"].map("{:,.2f}".format)
size_group_formatted["% Passing Math"] = size_group_formatted["% Passing Math"].map("{:.2%}".format)
size_group_formatted["% Passing Reading"] = size_group_formatted["% Passing Reading"].map("{:.2%}".format)
size_group_formatted["% Overall Passing"] = size_group_formatted["% Overall Passing"].map("{:.2%}".format)

size_group_formatted

Average Math Score Average Reading Score % Passing Math  \
School Size                                                                  
Small (<1000)                   83.83                 83.97         93.95%   
Medium (1000-2000)              83.37                 83.87         93.62%   
Large (2000-5000)               77.48                 81.20         68.65%   

                   % Passing Reading % Overall Passing  
School Size                                             
Small (<1000)                 96.04%            90.14%  
Medium (1000-2000)            96.77%            90.62%  
Large (2000-5000)             82.13%            56.57%

In [11]:
# Group By School Type Using the Weighted values (temp_df) from a previous cell

temp_df_type = temp_df.groupby(['School Type']).\
                        agg({"Total Students": ["sum"], 
                             "weighted_math": ["sum"], 
                             "weighted_reading": ["sum"], 
                             "weighted_math_pct": ["sum"],
                             "weighted_reading_pct": ["sum"], 
                             "weighted_overall_pct": ["sum"]}) 

#drop the aggregating function from the column name
temp_df_type.columns = temp_df_type.columns.droplevel(1)

#divide the weighted sums by the total students in each group to get the weighted averages
temp_df_type["weighted_math"] = (temp_df_type["weighted_math"]/temp_df_type["Total Students"])
temp_df_type["weighted_reading"] = (temp_df_type["weighted_reading"]/temp_df_type["Total Students"])
temp_df_type["weighted_math_pct"] = (temp_df_type["weighted_math_pct"]/temp_df_type["Total Students"])
temp_df_type["weighted_reading_pct"] = (temp_df_type["weighted_reading_pct"]/temp_df_type["Total Students"])
temp_df_type["weighted_overall_pct"] = (temp_df_type["weighted_overall_pct"]/temp_df_type["Total Students"])
    
#drop the total students column as it is no longer needed
temp_df_type = temp_df_type.drop(temp_df_type.columns[[ 0 ]], axis=1)

#rename the columns
temp_df_type = temp_df_type.rename(columns = {"weighted_math": "Average Math Score",
                                                 "weighted_reading": "Average Reading Score",
                                                 "weighted_math_pct": "% Passing Math",
                                                 "weighted_reading_pct": "% Passing Reading",
                                                 "weighted_overall_pct": "% Overall Passing",})
    
#copy the data frame to a new one for formatting
type_group_formatted = temp_df_type.copy()

#format the copied data frame
type_group_formatted["Average Reading Score"] = \
                                        type_group_formatted["Average Reading Score"].map("{:,.2f}".format)
type_group_formatted["Average Math Score"] = type_group_formatted["Average Math Score"].map("{:,.2f}".format)
type_group_formatted["% Passing Math"] = type_group_formatted["% Passing Math"].map("{:.2%}".format)
type_group_formatted["% Passing Reading"] = type_group_formatted["% Passing Reading"].map("{:.2%}".format)
type_group_formatted["% Overall Passing"] = type_group_formatted["% Overall Passing"].map("{:.2%}".format)

type_group_formatted

Average Math Score Average Reading Score % Passing Math  \
School Type                                                           
Charter                  83.41                 83.90         93.70%   
District                 76.99                 80.96         66.52%   

            % Passing Reading % Overall Passing  
School Type                                      
Charter                96.65%            90.56%  
District               80.91%            53.70%

In [12]:
# Note: The results below this match the sample results given but are technically incorrect as a weight average is 
#        needed since not all district schools are the exact same size (lickwise with charter schools)

temp_df_test = new_df.groupby(['School Type']).\
                        agg({"Average Math Score": ["mean"], 
                             "Average Reading Score": ["mean"], 
                             "% Passing Math": ["mean"],
                             "% Passing Reading": ["mean"], 
                             "% Overall Passing": ["mean"]}) 

temp_df_test.columns = temp_df_test.columns.droplevel(1)

temp_df_test["Average Reading Score"] = \
                                        temp_df_test["Average Reading Score"].map("{:,.2f}".format)
temp_df_test["Average Math Score"] = temp_df_test["Average Math Score"].map("{:,.2f}".format)
temp_df_test["% Passing Math"] = temp_df_test["% Passing Math"].map("{:.2%}".format)
temp_df_test["% Passing Reading"] = temp_df_test["% Passing Reading"].map("{:.2%}".format)
temp_df_test["% Overall Passing"] = temp_df_test["% Overall Passing"].map("{:.2%}".format)

temp_df_test

Average Math Score Average Reading Score % Passing Math  \
School Type                                                           
Charter                  83.47                 83.90         93.62%   
District                 76.96                 80.97         66.55%   

            % Passing Reading % Overall Passing  
School Type                                      
Charter                96.59%            90.43%  
District               80.80%            53.67%